In [ ]:
import darwinex_ticks as dtw1
import pandas as pd
import glob, os
import numpy as np
from datetime import date
from datetime import timedelta  

In [ ]:
pairs_list=list(pd.read_csv("C:/Users/Isabel Cristina/Desktop/Algo_trading_py/configs/pairs.txt", header = None)[0])
configs_info=pd.read_csv("C:/Users/Isabel Cristina/Desktop/Algo_trading_py/configs/configs.txt", header = None, sep= "=", names=['Desc', 'Value'])

user= configs_info[configs_info.Desc=='user'].Value.iloc[0]
password= configs_info[configs_info.Desc=='password'].Value.iloc[0]
host_name= configs_info[configs_info.Desc=='host_name'].Value.iloc[0]
start_date= configs_info[configs_info.Desc=='start_date'].Value.iloc[0]
end_date= configs_info[configs_info.Desc=='end_date'].Value.iloc[0]
percentile= pd.to_numeric(configs_info[configs_info.Desc=='percentile'].Value.iloc[0])

dwt = dtw1.DarwinexTicksConnection(dwx_ftp_user=user,
                       dwx_ftp_pass=password,
                       dwx_ftp_hostname=host_name,
                       dwx_ftp_port=21)
path = 'C:/Users/Isabel Cristina/Desktop/Algo_trading_py/data/historical_data/*.csv'
csv_files = glob.glob(path)

In [ ]:
date_end = date.today() - timedelta(days = 5)
for pair in pairs_list:
    
    file = [s for s in csv_files if pair in s ]

    if len(file) == 0:
        print("\n No previoius data downloaded")
        date_start = start_date
    elif first_dwn_data:
        date_start = start_date
    else:
        date_start = file[0].split("\\")[-1].split("_")[-1].split(".")[0]-timedelta(days = 1)

    date_range = pd.date_range(date_start,date_end , freq='D').to_series()
    
    data_ohlc = pd.DataFrame()
    
    for date_ini in date_range:

        date_end_period = pd.to_datetime(date_ini) + timedelta(hours = 23)

        print("Dowloading data from Darwinex, date: {} ".format(date_ini))
        try:
            data = dwt.ticks_from_darwinex(pair, start=date_ini, end=date_end_period,  darwinex_time=True)
        except:
            print("Date with not data")
        data['Price'] = (data.Ask + data.Bid) / 2
        data_ohlc_prev = data['Price'].resample('1Min').ohlc()
        data_ohlc =data_ohlc.append(data_ohlc_prev)

    data_ohlc.index = pd.to_datetime(data_ohlc.index.astype(str).map(lambda x: str(x)[:19]))
    min_date = min(data_ohlc.index)
    max_date = max(data_ohlc.index)

    if len(file) == 0:
        
        data_total = data_ohlc
        start_date_filter = min_date
    else:
        
        # Date for filtering
        start_date_filter = pd.to_datetime('2006-07-01')
        
        # Reading data from mt4
        print("Reading csv")
        data_mt = pd.read_csv(file, header=None)

        # Procesing data
        data_mt.columns = ["Date", "Hora", "open", "high", "low", "close"]#, "Vol"]
        data_mt['Time'] = pd.to_datetime(data_mt['Date'] + ' ' + data_mt['Hora'])
        data_mt = data_mt[['Time', 'open', 'high', 'low', 'close']].set_index('Time')

        # Setting time zone
        data_mt = data_mt.loc[start_date_filter:min_date]

        # Merging data
        data_total = data_mt.append(data_ohlc).reset_index()
    
    
    data_total['Date'] = data_total.Time.dt.strftime('%Y.%m.%d')
    data_total['Hour'] = data_total.Time.dt.strftime('%H:%M')
    data_total = data_total[['Date', 'Hour', 'open', 'high', 'low', 'close']]

    # Saving csv data
    print("Saving updated data...")
    data_total.to_csv("data\{}_M1_from_{}_to_{}.csv".format(pair, str(start_date_filter), str(max_date).split(sep=" ")[0]), 
                                                                            header=False, index=False)
    os.remove("data\{}".format(file))

In [ ]:
data_ohlc#.iloc[701:750]

In [ ]:
for file in csv_files:
    pair = file.split("\\")[-1][0:6]
    print("\n--- CREATING DATA FOR FILE {}: {} ---\n".format(i, pair))

    # Reading data from darwinex
    print("Dowloading data from Darwinex from {} to {}".format(start_date, end_date))
    data = dwt.ticks_from_darwinex(pair, start=start_date, end=end_date, verbose = False)
    data['Price'] = (data.Ask + data.Bid) / 2
    data_ohlc = data['Price'].resample('1Min').ohlc()

    # Date for filtering
    start_date_filter = pd.to_datetime('2006-07-01').tz_localize('UTC')
    min_date = min(data_ohlc.index)
    # min_hour = data_ohlc.Hour[0]
    max_date = max(data_ohlc.index)

    # Reading data from mt4
    print("Reading csv")
    data_mt = pd.read_csv(file, header=None)

    # Procesing data
    data_mt.columns = ["Date", "Hora", "open", "high", "low", "close"]#, "Vol"]
    data_mt['Time'] = pd.to_datetime(data_mt['Date'] + ' ' + data_mt['Hora'])
    data_mt = data_mt[['Time', 'open', 'high', 'low', 'close']].set_index('Time')

    # Setting time zone
    data_mt.index = data_mt.index.tz_localize('UTC')
    data_mt.index = data_mt.index - pd.Timedelta(hours=2)
    data_mt = data_mt.loc[start_date_filter:min_date]

    # Merging data
    data_total = data_mt.append(data_ohlc).reset_index()
    data_total['Date'] = data_total.Time.dt.strftime('%Y.%m.%d')
    data_total['Hour'] = data_total.Time.dt.strftime('%H:%M')
    data_total = data_total[['Date', 'Hour', 'open', 'high', 'low', 'close']]

    # Saving csv data
    print("Saving updated data...")
    data_total.to_csv("data_updated\{}_M1_from_2006-07-01_to_{}.csv".format(pair, str(max_date).split(sep=" ")[0]), 
                                                                            header=False, index=False)

    i += 1
    del data, data_ohlc, data_total, data_mt
    gc.collect()
print ("\n--- DONE ---")
